In [40]:
import math
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import timedelta
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.models import (NumeralTickFormatter, CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, LogColorMapper, Slider)
from bokeh.models.formatters import DatetimeTickFormatter

output_notebook()

Loading BokehJS ...

In [45]:
df = pd.read_csv('../data/counties/co-est2019-alldata.csv')
df = df.where(df['STNAME'] == 'California')
df = df.dropna()
df['COUNTY'] = df['COUNTY'].astype('int64')
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
190,40.0,4.0,9.0,6.0,0,California,California,37253956.0,37254519.0,37319502.0,...,-5.151429,1.276797,1.495016,1.649031,2.203551,1.984957,0.500044,-0.629909,-2.130954,-3.276681
191,50.0,4.0,9.0,6.0,1,California,Alameda County,1510271.0,1510258.0,1512986.0,...,-6.965670,5.134201,8.497481,10.150136,11.190365,10.411032,3.989666,-0.196911,-0.945009,-1.848964
192,50.0,4.0,9.0,6.0,3,California,Alpine County,1175.0,1175.0,1161.0,...,31.559964,-57.675244,15.433500,16.085791,-41.666667,2.781641,-22.598870,62.094532,-20.000000,32.461677
193,50.0,4.0,9.0,6.0,5,California,Amador County,38091.0,38091.0,37886.0,...,11.395076,-4.401490,-7.930660,-8.843796,6.680664,11.822607,14.531292,32.491640,24.867195,11.824602
194,50.0,4.0,9.0,6.0,7,California,Butte County,220000.0,220005.0,219949.0,...,-50.048384,-1.050181,3.461542,3.493706,6.968328,4.413730,6.840230,11.052323,7.768471,-49.590123


In [46]:
ca_counties_shapefile = gpd.read_file('../data/CA_Counties/CA_Counties_TIGER2016.shp')
ca_counties_shapefile['COUNTYFP'] = ca_counties_shapefile['COUNTYFP'].astype('int64') 
ca_counties_shapefile.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,91,00277310,06091,Sierra,Sierra County,06,H1,G4020,None,None,None,A,2468694587,23299110,+39.5769252,-120.5219926,POLYGON ((-13431319.75097945 4821511.426395644...
1,06,67,00277298,06067,Sacramento,Sacramento County,06,H1,G4020,472,40900,None,A,2499183617,76073827,+38.4500114,-121.3404409,POLYGON ((-13490651.47641084 4680831.603393857...
2,06,83,00277306,06083,Santa Barbara,Santa Barbara County,06,H1,G4020,None,42200,None,A,7084000598,2729814515,+34.5370572,-120.0399729,(POLYGON ((-13423116.77225655 4042044.14856006...
3,06,9,01675885,06009,Calaveras,Calaveras County,06,H1,G4020,None,None,None,A,2641820834,43806026,+38.1838996,-120.5614415,"POLYGON ((-13428575.483353 4627725.227535474, ..."
4,06,111,00277320,06111,Ventura,Ventura County,06,H1,G4020,348,37100,None,A,4773390489,945942791,+34.3587415,-119.1331432,(POLYGON ((-13317853.59433417 3931602.41399276...


In [47]:
list(ca_counties_shapefile)

['STATEFP',
 'COUNTYFP',
 'COUNTYNS',
 'GEOID',
 'NAME',
 'NAMELSAD',
 'LSAD',
 'CLASSFP',
 'MTFCC',
 'CSAFP',
 'CBSAFP',
 'METDIVFP',
 'FUNCSTAT',
 'ALAND',
 'AWATER',
 'INTPTLAT',
 'INTPTLON',
 'geometry']

In [48]:
counties = ca_counties_shapefile.merge(df, left_on='COUNTYFP', right_on='COUNTY')
counties.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
0,06,91,00277310,06091,Sierra,Sierra County,06,H1,G4020,None,...,6.664445,-32.921811,-7.137064,-6.890894,-10.666667,9.064966,-10.444744,16.798253,-2.667111,6.664445
1,06,67,00277298,06067,Sacramento,Sacramento County,06,H1,G4020,472,...,2.236311,1.686798,1.644560,2.590455,6.032211,6.873294,6.049268,5.615004,3.025394,4.204731
2,06,83,00277306,06083,Santa Barbara,Santa Barbara County,06,H1,G4020,None,...,-4.312552,-3.921458,3.585328,5.132477,3.994753,0.953166,-0.778281,-2.963003,-3.739291,-3.073030
3,06,9,01675885,06009,Calaveras,Calaveras County,06,H1,G4020,None,...,7.052171,-4.126760,-5.468186,0.782385,2.776347,8.947298,11.120095,10.571080,2.341895,7.379671
4,06,111,00277320,06111,Ventura,Ventura County,06,H1,G4020,348,...,-6.130515,-1.099568,-2.475332,-0.562318,-0.866443,-1.324567,-3.151569,-2.399245,-4.679511,-5.965240
